In [1]:
# Colab cell 1: install Java + PySpark (run once per session)
!apt-get update -qq
!apt-get install -y openjdk-17-jdk-headless -qq

# Install pyspark (pip)
!pip install -q pyspark

# Set JAVA_HOME for this session
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
print("JAVA_HOME set to:", os.environ["JAVA_HOME"])

# Quick verification
import pyspark
print("pyspark version:", pyspark.__version__)


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../openjdk-17-jdk-headless_17.0.17+10-1~22.04_amd64.deb ...
Unpacking openjdk-17-jdk-headless:amd64 (17.0.17+10-1~22.04) over (17.0.16+8~us1-0ubuntu1~22.04.1) ...
Preparing to unpack .../openjdk-17-jre-headless_17.0.17+10-1~22.04_amd64.deb ...
Unpacking openjdk-17-jre-headless:amd64 (17.0.17+10-1~22.04) over (17.0.16+8~us1-0ubuntu1~22.04.1) ...
Setting up openjdk-17-jre-headless:amd64 (17.0.17+10-1~22.04) ...
Installing new version of config file /etc/java-17-openjdk/security/default.policy ...
Installing new version of config file /etc/java-17-openjdk/security/java.security ...
Setting up openjdk-17-jdk-headless:amd64 (17.0.17+10-1~22.04) ...
JAVA_HOME set to: /usr/lib/jvm/java-17-openjdk-amd64
pyspark vers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
ZIP_PATH = "/content/drive/MyDrive/DatasetToCompleteTheSixSparkExercises.zip"   # <- change this
TARGET_DIR = "/content/data"                              # <- change if you want

# create target dir and unzip
!mkdir -p "{TARGET_DIR}"
!unzip -q "{ZIP_PATH}" -d "{TARGET_DIR}"

# show top-level of extracted folder
!ls -la "{TARGET_DIR}"

total 68
drwxr-xr-x 5 root root  4096 Dec 11 06:58 .
drwxr-xr-x 1 root root  4096 Dec 11 06:54 ..
drwxr-xr-x 2 root root  4096 Apr  7  2020 products_parquet
drwxr-xr-x 2 root root 53248 Apr  7  2020 sales_parquet
drwxr-xr-x 2 root root  4096 Apr  7  2020 sellers_parquet


In [4]:
# Colab cell 5: start Spark
from pyspark.sql import SparkSession

# adjust driver memory if needed; do not exceed available RAM
spark = SparkSession.builder \
    .appName("Pyspark_Colab_Assignment") \
    .master("local[*]") \
    .config("spark.driver.memory", "6g") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()

print("Spark session started. Version:", spark.version)


Spark session started. Version: 4.0.1


In [5]:
# Colab cell 6: load parquet folders directly
from pathlib import Path
BASE_DIR = Path("/content/data")   # <- change if needed

products_path = str(BASE_DIR / "products_parquet")
sales_path    = str(BASE_DIR / "sales_parquet")
sellers_path  = str(BASE_DIR / "sellers_parquet")

print("Paths used:")
print("products:", products_path)
print("sales   :", sales_path)
print("sellers :", sellers_path)

# Load (this will work on Colab/Linux even with many files)
products_df = spark.read.parquet(products_path)
sales_df    = spark.read.parquet(sales_path)
sellers_df  = spark.read.parquet(sellers_path)

# Show quick schema previews
print("=== products schema ===")
products_df.printSchema()
print("=== sales schema ===")
sales_df.printSchema()
print("=== sellers schema ===")
sellers_df.printSchema()


Paths used:
products: /content/data/products_parquet
sales   : /content/data/sales_parquet
sellers : /content/data/sellers_parquet
=== products schema ===
root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: string (nullable = true)

=== sales schema ===
root
 |-- order_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- num_pieces_sold: string (nullable = true)
 |-- bill_raw_text: string (nullable = true)

=== sellers schema ===
root
 |-- seller_id: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- daily_target: string (nullable = true)



In [6]:
from pyspark.sql import functions as F

# Products table
products_df = products_df.withColumn("product_id", F.col("product_id").cast("long")) \
                         .withColumn("price", F.col("price").cast("double"))

# Sales table
sales_df = (sales_df
            .withColumn("order_id", F.col("order_id").cast("long"))
            .withColumn("product_id", F.col("product_id").cast("long"))
            .withColumn("seller_id", F.col("seller_id").cast("long"))
            .withColumn("num_pieces_sold", F.col("num_pieces_sold").cast("long"))
            .withColumn("date", F.to_date("date", "yyyy-MM-dd"))
           )

# Sellers table
sellers_df = sellers_df.withColumn("seller_id", F.col("seller_id").cast("long")) \
                       .withColumn("daily_target", F.col("daily_target").cast("double"))

print("Schemas after casting:")
products_df.printSchema()
sales_df.printSchema()
sellers_df.printSchema()


Schemas after casting:
root
 |-- product_id: long (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: double (nullable = true)

root
 |-- order_id: long (nullable = true)
 |-- product_id: long (nullable = true)
 |-- seller_id: long (nullable = true)
 |-- date: date (nullable = true)
 |-- num_pieces_sold: long (nullable = true)
 |-- bill_raw_text: string (nullable = true)

root
 |-- seller_id: long (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- daily_target: double (nullable = true)



## Question 1

In [7]:
print("total no. of products  =",products_df.count())
print("total no. of sellers  =",sellers_df.count())
print("total no. of sales  =",sales_df.count())

total no. of products  = 75000000
total no. of sellers  = 10
total no. of sales  = 20000040


##  Question 2

In [8]:
from pyspark.sql import functions as F

# products sold at least once
products_sold = sales_df.select("product_id").distinct().count()
print("Products sold at least once:", products_sold)

# product appearing in the most distinct orders
product_order_counts = (sales_df
    .select("product_id", "order_id")
    .distinct()
    .groupBy("product_id")
    .agg(F.countDistinct("order_id").alias("num_orders"))
    .orderBy(F.desc("num_orders")))

product_order_counts.show(10, truncate=False)

# top product
top_product = product_order_counts.limit(1).collect()
print("Top product (product_id, num_orders):", top_product)


Products sold at least once: 993429
+----------+----------+
|product_id|num_orders|
+----------+----------+
|0         |19000000  |
|10978356  |3         |
|34681047  |3         |
|18182299  |3         |
|52606213  |3         |
|36269838  |3         |
|28592106  |3         |
|72017876  |3         |
|69790381  |3         |
|17944574  |3         |
+----------+----------+
only showing top 10 rows
Top product (product_id, num_orders): [Row(product_id=0, num_orders=19000000)]


## Question 3

In [9]:
products_df.printSchema()
sales_df.printSchema()
sellers_df.printSchema()

root
 |-- product_id: long (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: double (nullable = true)

root
 |-- order_id: long (nullable = true)
 |-- product_id: long (nullable = true)
 |-- seller_id: long (nullable = true)
 |-- date: date (nullable = true)
 |-- num_pieces_sold: long (nullable = true)
 |-- bill_raw_text: string (nullable = true)

root
 |-- seller_id: long (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- daily_target: double (nullable = true)



In [10]:
# join sales with product prices
sales_with_price = (sales_df
    .withColumn("num_pieces", F.col("num_pieces_sold").cast("long"))
    .join(products_df.select("product_id", "price"), on="product_id", how="left"))
    # .withColumn("line_revenue", F.col("price_num") * F.col("num_pieces"))
sales_with_price=sales_with_price.withColumn("line_revenue", F.col("price") * F.col("num_pieces"))
# order revenue of each order
order_revenue = (sales_with_price
    .groupBy("order_id")
    .agg(F.sum("line_revenue").alias("order_revenue")))

# average revenue (mean of order_revenue)
avg_revenue = order_revenue.agg(F.mean("order_revenue").alias("avg_order_revenue")).collect()[0]["avg_order_revenue"]
print("Average order revenue:", avg_revenue)

Average order revenue: 1246.1338560822878


## Question 4


In [11]:
order_with_revenue=sales_with_price.groupBy("order_id","seller_id")\
                                            .agg(F.sum("line_revenue").alias("order_seller_revenue"))
order_with_target = (order_with_revenue
                     .join(sellers_df.select("seller_id","daily_target"), on="seller_id", how="left"))


In [12]:
order_seller_with_pct = order_with_target.withColumn(
    "pct_of_daily_quota",
    F.when(F.col("daily_target").isNull() | (F.col("daily_target") == 0), F.lit(None))
     .otherwise((F.col("order_seller_revenue") / F.col("daily_target")) * 100)
)

seller_avg_pct = (order_seller_with_pct
    .groupBy("seller_id")
    .agg(
        F.mean("pct_of_daily_quota").alias("avg_pct_of_daily_quota"),
        F.count("order_id").alias("num_orders_considered")
    )
    .orderBy(F.desc("avg_pct_of_daily_quota"))
)

seller_avg_pct.show(200, truncate=False)

+---------+----------------------+---------------------+
|seller_id|avg_pct_of_daily_quota|num_orders_considered|
+---------+----------------------+---------------------+
|1        |1.4844178645806272    |110805               |
|3        |1.2318678193054733    |111328               |
|8        |0.6946060998564042    |110882               |
|2        |0.5064829818617154    |111233               |
|6        |0.3609385251748184    |111318               |
|5        |0.3170589299015169    |110874               |
|9        |0.2905299815673645    |111392               |
|4        |0.24841173704802258   |111168               |
|7        |0.19601246306317607   |111040               |
|0        |0.044437489776886434  |19000000             |
+---------+----------------------+---------------------+



## Question 5

In [13]:
print(products_df.columns,'\n')
print(sales_df.columns,'\n')
sellers_df.columns

['product_id', 'product_name', 'price'] 

['order_id', 'product_id', 'seller_id', 'date', 'num_pieces_sold', 'bill_raw_text'] 



['seller_id', 'seller_name', 'daily_target']

In [14]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# total pieces sold per product-seller
prod_seller_agg = (sales_df
    .withColumn("num_pieces", F.col("num_pieces_sold").cast("long"))
    .groupBy("product_id", "seller_id")
    .agg(F.sum("num_pieces").alias("total_pieces_sold"))
)

# rank sellers per product by descending total_pieces_sold
w_desc = Window.partitionBy("product_id").orderBy(F.desc("total_pieces_sold"), F.asc("seller_id"))
prod_seller_ranked = prod_seller_agg.withColumn("rank_desc", F.row_number().over(w_desc))

second_most_per_product = prod_seller_ranked.filter(F.col("rank_desc") == 2).select(
    "product_id", F.col("seller_id").alias("second_seller_id"), "total_pieces_sold"
)

# least selling: rank ascending
w_asc = Window.partitionBy("product_id").orderBy(F.asc("total_pieces_sold"), F.asc("seller_id"))
prod_seller_ranked_asc = prod_seller_agg.withColumn("rank_asc", F.row_number().over(w_asc))

least_seller_per_product = prod_seller_ranked_asc.filter(F.col("rank_asc") == 1).select(
    "product_id", F.col("seller_id").alias("least_seller_id"), "total_pieces_sold"
)

# show samples
print("Second-most selling sellers (sample):")
second_most_per_product.show(50, truncate=False)

print("Least-selling sellers (sample):")
least_seller_per_product.show(50, truncate=False)




Second-most selling sellers (sample):
+----------+----------------+-----------------+
|product_id|second_seller_id|total_pieces_sold|
+----------+----------------+-----------------+
|72638     |2               |27               |
|75721     |2               |3                |
|91069     |1               |60               |
|96545     |2               |62               |
|100142    |9               |47               |
|127612    |5               |2                |
|133613    |5               |41               |
|157889    |4               |86               |
|224335    |9               |11               |
|307787    |1               |62               |
|344440    |3               |63               |
|347070    |2               |15               |
|367935    |8               |7                |
|396156    |1               |4                |
|407839    |7               |60               |
|416887    |7               |10               |
|446702    |7               |21               |
|4

## Question 6

In [15]:
print("Second-most seller for product 0:")
second_most_per_product.filter(F.col("product_id") == 0).show(truncate=False)

print("Least-selling seller for product 0:")
least_seller_per_product.filter(F.col("product_id") ==0).show(truncate=False)

Second-most seller for product 0:
+----------+----------------+-----------------+
|product_id|second_seller_id|total_pieces_sold|
+----------+----------------+-----------------+
+----------+----------------+-----------------+

Least-selling seller for product 0:
+----------+---------------+-----------------+
|product_id|least_seller_id|total_pieces_sold|
+----------+---------------+-----------------+
|0         |0              |959445802        |
+----------+---------------+-----------------+



## Question 7

In [16]:
import hashlib
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf, col

def compute_hashed_bill(order_id_val, bill_text):
    # handle null bill_text
    if bill_text is None:
        return None

    # determine even/odd safely
    try:
        oid = int(order_id_val)
        is_even = (oid % 2 == 0)
    except Exception:
        is_even = False  # if the order id is null it will throw error and is_even will be treated as odd

    if is_even:
        # count capital 'A'
        count_A = bill_text.count('A')
        result = bill_text
        # apply MD5 iteratively count_A times
        for _ in range(count_A):
            result = hashlib.md5(result.encode('utf-8')).hexdigest()
        return result
    else:
        # odd -> sha256 once
        return hashlib.sha256(bill_text.encode('utf-8')).hexdigest()

compute_hashed_bill_udf = udf(compute_hashed_bill, StringType())

# create the new DataFrame with hashed_bill
sales_hashed = sales_df.withColumn("hashed_bill", compute_hashed_bill_udf(col("order_id"), col("bill_raw_text")))




In [17]:

sales_hashed.select("order_id", "bill_raw_text", "hashed_bill").show(20, truncate=80)

+--------+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|order_id|                                                                   bill_raw_text|                                                                     hashed_bill|
+--------+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|       1|kyeibuumwlyhuwksxodcawelubbyznxvpfxyxzhftudwtiemdhqaqoDeodltimrusmwtfdikxoqpu...|                f6fa2a8be04a4ead64b99befecb2a6c972693eada87f2e3c4df0d4080c691e39|
|       2|jfyuoyfkeyqkckwbubhpoxkghhhznurbodouqwsdychQqsejvjbcdnfvcktclasoablyocbstisye...|jfyuoyfkeyqkckwbubhpoxkghhhznurbodouqwsdychQqsejvjbcdnfvcktclasoablyocbstisye...|
|       3|uyjihlzhzcswxcccxiohemvmUzybmytqzdcvmuvzgtgsdjhdahqpknzmancicqdxweotpmezxqVVq...|                416376a64cd652e7b399cee99fed